This notebook produces plots for the results of using f5c's R10 model, with and without ReQuant, compared to Bonito calls.

In [ ]:
import bz2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
plt.rcParams['pdf.fonttype'] = 42


# Read in the methylation calls from bonito
def read_bonito_methcalls(path_methcalls):
    print('Loading:',path_methcalls)
    all_reads = {}

    with bz2.open(path_methcalls,mode='rb') as methcalls:
        header = methcalls.readline().decode().split()

        idx_read_id = 0
        idx_ref_pos = 2
        idx_ref_chr = 3
        idx_ref_mst = header.index('ref_mod_strand')
        idx_ref_mod = header.index('mod_qual')

        last_read = ''
        cur_read = []

        # print(header)

        split_line = []
        for line in methcalls:
            
            split_line = line.split()
            call_read  = split_line[idx_read_id].decode()+split_line[idx_ref_mst].decode()+split_line[idx_ref_chr].decode()
            call_pos   = int(split_line[idx_ref_pos])
            call_modq  = float(split_line[idx_ref_mod])

            if call_read == last_read:
                if call_pos != -1:
                    cur_read.append((call_pos,call_modq))
            else:
                if cur_read != []:
                    sorted_calls = sorted(cur_read)
                    all_reads[last_read+'_p'] = np.array([x[0] for x in sorted_calls])
                    all_reads[last_read+'_m'] = np.array([x[1] for x in sorted_calls])
                last_read = call_read
                cur_read = []
            # if len(all_reads) > 10000:
            #     break
    return all_reads


# Read in the methylation calls from bonito
def read_f5c_methcalls(path_methcalls, header=False):
    print('Loading:',path_methcalls)
    all_reads = {}

    with bz2.open(path_methcalls,mode='rb') as methcalls:
        if header:
            header = methcalls.readline().decode().split()
            # print(header)

        idx_read_id = 4
        idx_ref_pos_s = 2
        idx_ref_pos_e = 3
        idx_ref_chr = 0
        idx_ref_mst = 1
        idx_ref_mod = 5

        last_read = ''
        cur_read = []

        # print(header)

        split_line = []
        for line in methcalls:
            
            split_line = line.split()
            call_read  = split_line[idx_read_id].decode()+split_line[idx_ref_mst].decode()+split_line[idx_ref_chr].decode()
            call_pos_s = int(split_line[idx_ref_pos_s])
            call_pos_e = int(split_line[idx_ref_pos_e])
            call_modq  = float(split_line[idx_ref_mod])

            if call_read == last_read:
                cur_read.append((call_pos_s,call_pos_e,call_modq))
            else:
                if cur_read != []:
                    sorted_calls = sorted(cur_read)
                    all_reads[last_read+'_p'] = np.array([x[0:2] for x in sorted_calls])
                    all_reads[last_read+'_m'] = np.array([x[2] for x in sorted_calls])
                last_read = call_read
                cur_read = []
            # if len(all_reads) > 10000:
            #     break
    return all_reads


# Match calls made from bonito to f5c calls
def match_calls(bonito_calls, f5c_calls):
    print('Matching calls')
    matched_calls = []
    for read in f5c_calls:
        if read.endswith('p') and read in bonito_calls:
            # print(read)

            bonito_p  = bonito_calls[read]

            # Looks like like there's a 1 bp difference between f5c and bonito
            # output if case of reverse stranded calls, fix that by shifting 1bp
            if read[36] == '-': # 36 is the index of the strand in the read name
                bonito_p = bonito_p-1

            f5c_cp    = f5c_calls[read]

            bonito_m  = bonito_calls[read[:-1]+'m']
            f5c_cm    = f5c_calls[read[:-1]+'m']

            # print(bonito_p[:5],f5c_cp[:5])

            f5c_i, f5c_p = 0, f5c_cp[0]
            for i,pos in enumerate(bonito_p):
                # print('b_pos:',pos,'f_pos:',f5c_p)
                # Make f5c index catch up to bonito positions
                while f5c_p[1] < pos and f5c_i < len(f5c_cp)-1:
                    f5c_i += 1
                    f5c_p = f5c_cp[f5c_i]
                    # print('\tb_pos:',pos,'f_pos:',f5c_p)
                # Check if bonito call is within f5c call range
                if pos < f5c_p[0] or pos > f5c_p[1]:
                    # print(pos,f5c_p,bonito_m[i],f5c_cm[f5c_i],'no')
                    continue
                # Handle a match
                else:
                    # print(pos,f5c_p,bonito_m[i],f5c_cm[f5c_i])
                    matched_calls.append((bonito_m[i],f5c_cm[f5c_i]))

                # if i > 10:
                #     break
    return np.array(matched_calls)

def compare_results(matched_calls):
    print('Matched calls:',matched_calls.shape[0])
    matched_calls = np.array(matched_calls).T
    matched_calls[0] = matched_calls[0]-0.5
    pos_bon  = np.sum(np.greater(matched_calls[0]>0,matched_calls[1]>0))/matched_calls.shape[1]
    pos_f5c  = np.sum(np.less(matched_calls[0]>0,matched_calls[1]>0))/matched_calls.shape[1]
    pos_both = np.sum(np.logical_and(matched_calls[0]>0,matched_calls[1]>0))/matched_calls.shape[1]
    neg_both = np.sum(np.logical_and(matched_calls[0]<0,matched_calls[1]<0))/matched_calls.shape[1]
    print('Bonito only:\t',pos_bon,'\nf5c only:\t',pos_f5c,'\nBoth:\t\t',pos_both,'\nNeither:\t\t',neg_both)
    print('Agreement:\t',pos_both+neg_both,'\nDisagreement:\t',pos_bon+pos_f5c)
    return matched_calls.shape[1],pos_bon,pos_f5c,pos_both,neg_both,pos_both+neg_both,pos_bon+pos_f5c

In [ ]:

bonito_calls = read_bonito_methcalls('data/f5c/bonito_calls_chr22.tsv.bz2')
f5c_calls = read_f5c_methcalls('data/f5c/PAM63167_result_chr22.tsv.bz2')
req_calls = read_f5c_methcalls('data/f5c/PAM63167_result_requant_chr22.tsv.bz2')

matched_calls_f5c = match_calls(bonito_calls,f5c_calls)
matched_calls_req = match_calls(bonito_calls,req_calls)

# print(matched_calls)
# matched_calls = [x for x in matched_calls if abs(x[1]) > 2]
# plt.plot([x[1] for x in matched_calls_f5c],[x[0] for x in matched_calls_f5c],'o',alpha=0.1)
# plt.figure()
# plt.plot([x[1] for x in matched_calls_req],[x[0] for x in matched_calls_req],'o',alpha=0.1)

# matched_calls = np.array(matched_calls).T
# matched_calls[0] = matched_calls[0]-0.5
# matched_same = np.sum(matched_calls>0,axis=0)
# print(np.sum(matched_same==0),np.sum(matched_same==1),np.sum(matched_same==2))
# matched_calls[0]>0.5 + matched_calls[1]>0

In [ ]:

print('\nf5c by itself:')
compare_results(matched_calls_f5c)
print('\nf5c after requant:')
compare_results(matched_calls_req)
# Looks like after requant the frequency of calls in accordance drops by 1.5% in total



In [ ]:
# Compare again but after filtering all f5c calls with abs(LLR) < 2
print('\nf5c by itself:')
compare_results(matched_calls_f5c[np.abs(matched_calls_f5c.T[1])>=2])
print('\nf5c after requant:')
compare_results(matched_calls_req[np.abs(matched_calls_req.T[1])>=2])
# Looks like this drops the difference in agreements to 0.3%,
# f5c direct went up to 98% agreement with bonito,
# requants version 98,7%

In [ ]:
# Now again, after training a model on chr22 and applying it to chr21 instead

bonito_calls = read_bonito_methcalls('data/f5c/bonito_calls_chr21.tsv.bz2')
direct_calls = read_f5c_methcalls('data/f5c/PAM63167_result_requant_direct.tsv.bz2',header=True)
added_calls = read_f5c_methcalls('data/f5c/PAM63167_result_requant_added.tsv.bz2',header=True)
replaced_calls = read_f5c_methcalls('data/f5c/PAM63167_result_requant_replaced.tsv.bz2',header=True)

matched_calls_direct   = match_calls(bonito_calls,direct_calls)
matched_calls_added    = match_calls(bonito_calls,added_calls)
matched_calls_replaced = match_calls(bonito_calls,replaced_calls)

In [ ]:
# test_read = '6db38711-4122-4f21-8038-cc373de768ee+chr21_p'
# bonito_calls[test_read]
# direct_calls[test_read]

# Only 40031 out of 9025068 bonito calls are withing 0.4-0.6, so filtering that seems unlikely to influence results
print(matched_calls_direct.shape[0]-sum(matched_calls_direct.T[0]>0.6)-sum(matched_calls_direct.T[0]<0.4))

In [ ]:

file_model = './data/f5c/r10_450bps.cpg.9mer.template.round4.model'
# file_model = './data/f5c/rq_test.added.model'
# file_model = './data/f5c/rq_test.replaced.model'

kmers_canonical = {}
kmers_modified = {}
kmers_trained = set()

with open(file_model,'r') as fh_model:
    for line in fh_model:
        if line.startswith('#'):
            continue
        split_line = line.split()
        if split_line[0].count('M') <= 1:
            if 'CG' in split_line[0]:
                kmers_canonical[split_line[0]] = float(split_line[1])
            elif 'MG' in split_line[0]:
                kmers_modified[split_line[0]] = float(split_line[1])

for kmer,val in kmers_modified.items():
    if val != kmers_canonical[kmer.replace('M','C')]:
        kmers_trained.add(kmer.replace('M','C'))

def check_seq(ref_seq):
    # print(ref_seq)
    trained_sum = 0
    for i in range(len(ref_seq)-8):
        trained_sum += (ref_seq[i:i+9] in kmers_trained)
    # print(ref_seq,trained_sum/(len(ref_seq)-8))
    return trained_sum/(len(ref_seq)-8)

# Read in the methylation calls from bonito
def read_f5c_methcalls_filt(path_methcalls, header=False):
    print('Loading:',path_methcalls)
    all_reads = {}

    with bz2.open(path_methcalls,mode='rb') as methcalls:
        if header:
            header = methcalls.readline().decode().split()
            # print(header)

        idx_read_id = 4
        idx_ref_pos_s = 2
        idx_ref_pos_e = 3
        idx_ref_chr = 0
        idx_ref_mst = 1
        idx_ref_mod = 5
        idx_ref_seq = 10

        last_read = ''
        cur_read = []

        # print(header)

        split_line = []
        for line in methcalls:
            
            split_line = line.split()
            call_read  = split_line[idx_read_id].decode()+split_line[idx_ref_mst].decode()+split_line[idx_ref_chr].decode()
            call_pos_s = int(split_line[idx_ref_pos_s])
            call_pos_e = int(split_line[idx_ref_pos_e])
            call_modq  = float(split_line[idx_ref_mod])

            if check_seq(split_line[idx_ref_seq].decode()) > 0:
                continue

            if call_read == last_read:
                cur_read.append((call_pos_s,call_pos_e,call_modq))
            else:
                if cur_read != []:
                    sorted_calls = sorted(cur_read)
                    all_reads[last_read+'_p'] = np.array([x[0:2] for x in sorted_calls])
                    all_reads[last_read+'_m'] = np.array([x[2] for x in sorted_calls])
                last_read = call_read
                cur_read = []
            # if len(all_reads) > 10000:
            #     break
    return all_reads


print('\nf5c direct:')
direct_calls_filt   = read_f5c_methcalls_filt('data/f5c/PAM63167_result_requant_direct.tsv.bz2',   header=True)
matched_calls_filt_direct = match_calls(bonito_calls,direct_calls_filt)
# print('Any LLR:')
# compare_results(matched_calls_filt_direct)
# print('LLR>=2:')
# compare_results(matched_calls_filt_direct[np.abs(matched_calls_filt_direct.T[1])>=2])

print('\nf5c added:')
added_calls_filt    = read_f5c_methcalls_filt('data/f5c/PAM63167_result_requant_added.tsv.bz2',    header=True)
matched_calls_filt_added    = match_calls(bonito_calls,added_calls_filt)
# print('Any LLR:')
# compare_results(matched_calls_filt_added)
# print('LLR>=2:')
# compare_results(matched_calls_filt_added[np.abs(matched_calls_filt_added.T[1])>=2])

print('\nf5c replaced:')
replaced_calls_filt = read_f5c_methcalls_filt('data/f5c/PAM63167_result_requant_replaced.tsv.bz2', header=True)
matched_calls_filt_replaced = match_calls(bonito_calls,replaced_calls_filt)
# print('Any LLR:')
# compare_results(matched_calls_filt_replaced)
# print('LLR>=2:')
# compare_results(matched_calls_filt_replaced[np.abs(matched_calls_filt_replaced.T[1])>=2])

In [ ]:
# # Compare without filtering f5 calls
# print('\nf5c direct:')
# compare_results(matched_calls_direct)
# print('\nf5c added:')
# compare_results(matched_calls_added)
# print('\nf5c replaced:')
# compare_results(matched_calls_replaced)
results = dict()
for i in [0,0.5,1,1.5,2,2.5,3]:
    print('\n\n'+str(i))
    # Compare again but after filtering all f5c calls with abs(LLR) < 2
    print('\nf5c direct:')
    result = compare_results(matched_calls_direct[np.abs(matched_calls_direct.T[1])>=i])
    results['direct_'+str(i)+'_100'] = result
    result = compare_results(matched_calls_filt_direct[np.abs(matched_calls_filt_direct.T[1])>=i])
    results['direct_'+str(i)+'_0'] = result
    print('\nf5c added:')
    result = compare_results(matched_calls_added[np.abs(matched_calls_added.T[1])>=i])
    results['added_'+str(i)+'_100'] = result
    result = compare_results(matched_calls_filt_added[np.abs(matched_calls_filt_added.T[1])>=i])
    results['added_'+str(i)+'_0'] = result
    print('\nf5c replaced:')
    result = compare_results(matched_calls_replaced[np.abs(matched_calls_replaced.T[1])>=i])
    results['replaced_'+str(i)+'_100'] = result
    result = compare_results(matched_calls_filt_replaced[np.abs(matched_calls_filt_replaced.T[1])>=i])
    results['replaced_'+str(i)+'_0'] = result

In [ ]:
matched_calls_replaced

In [ ]:
from matplotlib.ticker import FormatStrFormatter
# print(results)

    # print('Bonito only:\t',pos_bon,'\nf5c only:\t',pos_f5c,'\nBoth:\t\t',pos_both,'\nNeither:\t\t',neg_both)
    # print('Agreement:\t',pos_both+neg_both,'\nDisagreement:\t',pos_bon+pos_f5c)
df_results = pd.DataFrame(results).T
df_results.columns=['Matched Calls','Bonito only','5c only','Both','Neither','Agreement','Disagreement']

#split df_results index into two columns using _ as delimiter
# df_results[['Tool','LLR']] = df_results.index.str.split("_",expand=True)
# df_results

split_index = df_results.index.str.split("_",expand=True)
df_results['Model'] = [x[0] for x in split_index]
df_results['LLR'] = [float(x[1]) for x in split_index]
df_results['Kmer Filt'] = [int(x[2]) for x in split_index]

# df_results['Model'].replace({'replaced':'polished'},inplace=True)
# Rename model imputation types
df_results.replace({'Model': {'direct':'not-imputed','added':'missing-imputed','replaced':'fully-imputed'}},inplace=True)


df_results['Matched Freq'] = df_results['Matched Calls']/df_results['Matched Calls'].loc['direct_0_100']

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['figure.figsize'] = 5,5
# sns.scatterplot(data=df_results[df_results['Kmer Filt']==100],x='LLR',y='Agreement',hue='Model')

plt.figure()
plt.axvline(x=2, color='k', linestyle='--', linewidth=1)
sns.lineplot(data=df_results[df_results['Kmer Filt']==100],x='LLR',y='Agreement',hue='Model',markers=True, style="Model")
plt.title('R10 agreement with Bonito, all sites')
plt.ylabel('Fraction of calls in agreement')
plt.xlabel('LLR Cutoff')
# plt.ylim(0.725,0.975)
plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
plt.tight_layout()
plt.savefig('./data/f5c/'+'f5c_bonito_all.pdf')
plt.figure()
plt.axvline(x=2, color='k', linestyle='--', linewidth=1)
sns.lineplot(data=df_results[df_results['Kmer Filt']==100],x='LLR',y='Matched Freq',hue='Model',markers=True, style="Model")
plt.title('R10 calls after filtering by LLR, all sites')
plt.ylabel('Fraction of calls retained')
plt.xlabel('LLR Cutoff')
plt.tight_layout()
plt.savefig('./data/f5c/'+'f5c_bonito_all_kept.pdf')
plt.figure()

df_simple = df_results[df_results['Model']!='missing-imputed'].copy()
df_simple['Model'].replace({'fully-imputed':'missing-/fully-imputed'},inplace=True)
df_simple['Matched Freq'] = df_simple['Matched Calls']/df_simple['Matched Calls'].loc['direct_0_0']
plt.axvline(x=2, color='k', linestyle='--', linewidth=1)
sns.lineplot(data=df_simple[df_simple['Kmer Filt']==0],x='LLR',y='Agreement',hue='Model',markers=True, style="Model")
plt.title('R10 agreement with Bonito, untrained 9-mers')
plt.ylabel('Fraction of calls in agreement')
plt.xlabel('LLR Cutoff')
plt.tight_layout()
plt.savefig('./data/f5c/'+'f5c_bonito_untr.pdf')
plt.figure()
plt.axvline(x=2, color='k', linestyle='--', linewidth=1)
sns.lineplot(data=df_simple[df_simple['Kmer Filt']==0],x='LLR',y='Matched Freq',hue='Model',markers=True, style="Model")
plt.title('R10 calls after filtering by LLR, untrained 9-mers')
plt.ylabel('Fraction of calls retained')
plt.xlabel('LLR Cutoff')
plt.tight_layout()
plt.savefig('./data/f5c/'+'f5c_bonito_untr_kept.pdf')
df_results['Model'].replace({'missing-/fully-imputed':'fully-imputed'},inplace=True)
sns.scatterplot(data=df_results[df_results['Kmer Filt']==0],x='LLR',y='Agreement',hue='Model')

# df_results[df_results['Kmer Filt']==0].groupby('Model').describe()
df_results